This is an example of a simple CNN developed, trained and utilized

AI was used to help generate the codebase

Note: Make sure that the tensorflow package is installed in your device.

In [72]:
# Lib imports
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import numpy as np

In [73]:
# DATASET DIRECTORY CONFIGURATION
# Download and unzip the dataset from Kaggle, set the directory paths accordingly.
train_dir = "./train"  # e.g. './muffin-vs-chihuahua/train'
test_dir = "./test"    # e.g. './muffin-vs-chihuahua/test'

In [74]:
# IMAGE PARAMETERS
# Used to resize the input images, also will determine the input size of your input layer.
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

In [75]:
# DATA PREPROCESSING & AUGMENTATION
# Optional but recommended for image processing tasks, especially with limited data.
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)
val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

Found 3788 images belonging to 2 classes.
Found 945 images belonging to 2 classes.
Found 945 images belonging to 2 classes.
Found 1184 images belonging to 2 classes.
Found 1184 images belonging to 2 classes.


In [76]:
# SIMPLE CNN MODEL ARCHITECTURE

# Added L2 regularization and activated dropout layers to mitigate overfitting
initial_learning_rate = 0.001
l2_reg = tf.keras.regularizers.l2(0.001)
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=10000,
    decay_rate=0.9,
    staircase=True
)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2_reg, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),
    layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2_reg),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),
    layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2_reg),
    layers.MaxPooling2D(2, 2),
    layers.Dropout(0.25),
    layers.Flatten(),
    layers.Dense(128, activation='relu', kernel_regularizer=l2_reg),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid', kernel_regularizer=l2_reg)
])

In [77]:
# Configure the model optimizers, loss function, and metrics
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [78]:
# TRAINING THE CNN
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

Epoch 1/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 60s 484ms/step - accuracy: 0.6885 - loss: 0.7917 - val_accuracy: 0.8127 - val_loss: 0.5681
Epoch 2/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 60s 484ms/step - accuracy: 0.6885 - loss: 0.7917 - val_accuracy: 0.8127 - val_loss: 0.5681
Epoch 2/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 68s 575ms/step - accuracy: 0.7978 - loss: 0.5497 - val_accuracy: 0.8042 - val_loss: 0.4882
Epoch 3/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 68s 575ms/step - accuracy: 0.7978 - loss: 0.5497 - val_accuracy: 0.8042 - val_loss: 0.4882
Epoch 3/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 80s 673ms/step - accuracy: 0.7949 - loss: 0.5131 - val_accuracy: 0.8444 - val_loss: 0.4358
Epoch 4/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 80s 673ms/step - accuracy: 0.7949 - loss: 0.5131 - val_accuracy: 0.8444 - val_loss: 0.4358
Epoch 4/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 58s 486ms/step - accuracy: 0.8271 - loss: 0.4754 - val_accuracy: 0.8677 - val_loss: 0.3963
Epoch 5/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 58s 486ms/step - accuracy: 0.8271 - loss: 0

In [79]:
# EVALUATE THE MODEL
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc}")

37/37 ━━━━━━━━━━━━━━━━━━━━ 8s 216ms/step - accuracy: 0.8328 - loss: 0.4230
37/37 ━━━━━━━━━━━━━━━━━━━━ 8s 216ms/step - accuracy: 0.8328 - loss: 0.4230
Test Accuracy: 0.8327702879905701
Test Accuracy: 0.8327702879905701


In [ ]:
# SAVE THE MODEL
model.save('exercise_6_trained_model_improved.h5')

In [81]:
# SIMPLE INFERENCE SCRIPT
from tensorflow.keras.preprocessing import image

def predict_image(img_path, model_path='muffin_vs_chihuahua_cnn.h5'):
    # Load without compiling to avoid metrics warning during inference-only usage
    model = tf.keras.models.load_model(model_path, compile=False)
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    pred = model.predict(img_array)[0,0]
    label = "Chihuahua" if pred >= 0.5 else "Muffin"
    print(f"Prediction: {label} (confidence: {pred:.2f})")


In [90]:
# Example usage:
predict_image("test/chihuahua/img_0_516.jpg")
predict_image("test/muffin/img_0_570.jpg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step


Prediction: Muffin (confidence: 0.03)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
Prediction: Chihuahua (confidence: 0.87)
